In [37]:
import os
import pandas as pd
import pickle
import json


from src.dataset.dataset_info import datasets, DatasetInfo
from src.dataset.clean_dataset import clean_dataset
from src.dataset.create_class_num_col import one_dataset_class_num_col
from src.dataset.features_analysis import feature_analysis_pipeline
from src.dataset.undersample_classes import undersample_classes
from src.graph.graph_measures import calculate_graph_measures
from local_variables import datasets_main_path


In [38]:
with_sort_timestamp = False
with_undersample_classes = False

# name = "cic_ids_2017_5_percent"
# original_path = "./testing_dfs/cic_ids_2017_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot_5_percent"
# original_path = "./testing_dfs/cic_ton_iot_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot"
# original_path = datasets_main_path + "cic_ton_iot.parquet"
# file_type = "parquet"

# name = "cic_ids_2017"
# original_path = datasets_main_path + "cic_ids_2017.parquet"
# file_type = "parquet"

name = "cic_bot_iot"
original_path = datasets_main_path + "cic_bot_iot.parquet"
file_type = "parquet"

# name = "cic_ton_iot_modified"
# original_path = datasets_main_path + "CIC-ToN-IoT-Modified.pkl"
# file_type = "pkl"

# name = "nf_ton_iotv2_modified"
# original_path = datasets_main_path + "NF-ToN-IoT-v2-Modified.pkl"
# file_type = "pkl"

# name = "ccd_inid_modified"
# original_path = datasets_main_path + "CCD-INID-Modified.pkl"
# file_type = "pkl"

# name = "nf_uq_nids_modified"
# original_path = datasets_main_path + "NF-UQ-NIDS-Modified.pkl"
# file_type = "pkl"


# name = "nf_bot_iot"

# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

folder_path = os.path.join("datasets", name)
output_path = os.path.join(folder_path, name + ".parquet")

if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

print(f"==>> original_path: {original_path}")
print(f"==>> folder_path: {folder_path}")
print(f"==>> output_path: {output_path}")


==>> original_path: c:\Users\Administrateur\Desktop\datasets\cic_bot_iot.parquet
==>> folder_path: datasets\cic_bot_iot
==>> output_path: datasets\cic_bot_iot\cic_bot_iot.parquet


# Preparing Datasets

### Reading and Cleaning

In [39]:
if file_type == "parquet":
    df = pd.read_parquet(original_path)
if file_type == "csv":
    df = pd.read_csv(original_path)
if file_type == "pkl":
    df = pd.read_pickle(original_path, compression="zip")

In [40]:
pd.options.display.max_columns = df.shape[1]

In [41]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack
0,192.168.100.150-192.168.100.3-54114-44581-6,192.168.100.150,54114.0,192.168.100.3,44581.0,6.0,21/05/2018 08:34:13 PM,64465.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,31.024587,6.446500e+04,0.000000e+00,64465.0,64465.0,0.0,0.000000e+00,0.0,0.0,64465.0,64465.0,0.0,64465.0,64465.0,0.0,0.0,0.0,0.0,0.0,44.0,0.000000,31.024587,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance
1,192.168.100.147-192.168.100.7-1649-80-6,192.168.100.147,1649.0,192.168.100.7,80.0,6.0,04/06/2018 05:38:04 PM,22646986.0,3.0,1.0,300.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,100.0,0.000000,17.662394,0.176624,7.548995e+06,5.561972e+06,13839423.0,3281987.0,19364999.0,9.682500e+06,13839423.0,5525576.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,20.0,0.132468,0.044156,100.0,100.0,100.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,300.0,1.0,100.0,-1.0,512.0,3.0,0.0,3281987.0,0.000000e+00,3281987.0,3281987.0,9682499.5,5.878778e+06,13839423.0,5525576.0,1,DDoS
2,192.168.100.148-192.168.100.3-59032-80-6,192.168.100.148,59032.0,192.168.100.3,80.0,6.0,04/06/2018 01:41:34 PM,153463.0,1.0,2.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,50.0,70.710678,651.622867,19.548686,7.673150e+04,7.671755e+04,130979.0,22484.0,0.0,0.000000e+00,0.0,0.0,22484.0,22484.0,0.0,22484.0,22484.0,0.0,0.0,0.0,0.0,20.0,44.0,6.516229,13.032457,0.0,100.0,50.0,57.735027,3333.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,66.666667,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,100.0,-1.0,29200.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance
3,192.168.100.149-192.168.100.7-2820-80-17,192.168.100.149,2820.0,192.168.100.7,80.0,17.0,04/06/2018 02:29:28 PM,24915470.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.200679,6.228868e+06,2.337803e+06,9613711.0,4568562.0,20138439.0,6.712813e+06,9613711.0,4568562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,8.0,0.160543,0.040136,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,4777031.0,0.000000e+00,4777031.0,4777031.0,7784938.5,2.586275e+06,9613711.0,5956166.0,1,DDoS
4,192.168.100.150-192.168.100.5-5146-80-17,192.168.100.150,5146.0,192.168.100.5,80.0,17.0,04/06/2018 06:17:12 PM,46437692.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.193808,5.804712e+06,3.854238e+06,14796053.0,3089320.0,42448141.0,6.064020e+06,14796053.0,3089320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,8.0,0.172274,0.021534,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,9597806.5,7.235516e+06,14714089.0,4481524.0,9080693.0,5.048133e+06,147

In [42]:
timestamp_format = "mixed"
# timestamp_format="%d/%m/%Y %I:%M:%S %p"

In [43]:
df.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
       'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std',
       'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt',
       'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count',
       'ECE Flag Cnt', 'Dow

In [44]:
list(df.dtypes)

[dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64')

In [45]:
df.describe(include="all")

c:\Users\Administrateur\Desktop\GNN-NIDS\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\Administrateur\Desktop\GNN-NIDS\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack
count,13428595,13428595,1.342860e+07,13428595,1.342860e+07,1.342860e+07,13428595,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,13428595.0,13428595.0,13428595.0,13428595.0,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595
unique,3801165,65,NaN,289,NaN,NaN,52577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
top,192.168.100.3-192.168.100.6-80-80-6,192.168.100.149,NaN,192.168.100.3,NaN,NaN,22/05/2018 10:34:03 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DDoS
freq,970,3444781,NaN,6519604,NaN,NaN,38919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4913917
mean,NaN,NaN,3.628175e+04,NaN,3.083070e+03,1.052028e+01,NaN,2.392739e+07,6.244988e+00,1.941958e+00,1.139214e+03,5.868170e+02,4.891711e+01,3.014244e+01,3.967568e+01,1.021963e+01,4.950581e+01,2.472766e+01,3.422074e+01,1.493871e+01,inf,inf,5.043371e+06,2.544748e+06,8.632454e+06,2.982927e+06,1.868874e+07,5.385607e+06,7.609911e+06,3.626096e+06,4.194108e+06,2.680705e+06,1.432846e+06,3.704321e+06,1.581196e+06,0.0,3.249037e-04,0.0,5.138289e-05,7.798760e+01,3.397108e+01,6.972033e+02,2.864551e+03,2.459191e+01,5.111406e+01,3.856766e+01,1.232099e+01,2.093779e+03,1.846358e-03,5.767217e-01,1.988220e-03,3.249037e-04,1.009651e-02,5.138289e-05,1.891486e-05,1.906380e-05,1.508969e-01,4.718481e+01,3.967568e+01,3.422074e+01,0

In [46]:
datesetInfo =   DatasetInfo(name="cic_bot_iot",
                path="datasets/cic_bot_iot/cic_bot_iot.parquet",
                file_type="parquet",
                src_ip_col="Src IP",
                src_port_col="Src Port",
                dst_ip_col="Dst IP",
                dst_port_col="Dst Port",
                flow_id_col="Flow ID",
                timestamp_col="Timestamp",
                label_col="Label",
                class_col="Attack",
                class_num_col="Class",
                timestamp_format="mixed",
                drop_columns=["Flow ID", "Src IP", "Dst IP",
                              "Timestamp", "Src Port", "Dst Port", "Attack"],
                )

In [47]:
df.shape

(13428595, 84)

In [48]:
# import numpy as np
# df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [49]:
df.describe()

c:\Users\Administrateur\Desktop\GNN-NIDS\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\Administrateur\Desktop\GNN-NIDS\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,13428595.0,13428595.0,13428595.0,13428595.0,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,13428595.0,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07,1.342860e+07
mean,3.628175e+04,3.083070e+03,1.052028e+01,2.392739e+07,6.244988e+00,1.941958e+00,1.139214e+03,5.868170e+02,4.891711e+01,3.014244e+01,3.967568e+01,1.021963e+01,4.950581e+01,2.472766e+01,3.422074e+01,1.493871e+01,inf,inf,5.043371e+06,2.544748e+06,8.632454e+06,2.982927e+06,1.868874e+07,5.385607e+06,7.609911e+06,3.626096e+06,4.194108e+06,2.680705e+06,1.432846e+06,3.704321e+06,1.581196e+06,0.0,3.249037e-04,0.0,5.138289e-05,7.798760e+01,3.397108e+01,6.972033e+02,2.864551e+03,2.459191e+01,5.111406e+01,3.856766e+01,1.232099e+01,2.093779e+03,1.846358e-03,5.767217e-01,1.988220e-03,3.249037e-04,1.009651e-02,5.138289e-05,1.891486e-05,1.906380e-05,1.508969e-01,4.718481e+01,3.967568e+01,3.422074e+01,0.0,0.0,0.0,0.0,0.0,0.0,6.244988e+00,1.139214e+03,1.941958e+00,5.868170e+02,-1.0,4.367415e+03,2.560740e+00,0.0,2.057990e+06,1.024662e+05,2.131767e+06,1.985147e+06,7.059793e+06,7.965339e+05,7.841705e+06,6.423615e+06,9.933540e-01
std,1.837759e+04,1.077831e+04,5.412621e+00,1.694607e+07,7.032073e+01,4.669613e+01,1.067439e+05,1.348631e+05,9.352407e+01,4.703005e+01,5.902777e+01,3.852885e+01,1.017514e+02,4.526063e+01,6.004179e+01,4.113049e+01,NaN,NaN,4.717544e+06,3.535748e+06,7.754256e+06,4.338853e+06,1.473655e+07,5.208623e+06,7.414981e+06,4.799812e+06,1.086254e+07,7.039556e+06,5.021360e+06,9.384352e+06,6.143070e+06,0.0,1.802216e-02,0.0,7.168002e-03,1.170441e+03,8.248526e+02,2.354403e+04,3.522637e+04,4.356298e+01,1.185539e+02,5.446863e+01,4.406781e+01,8.231524e+05,4.292959e-02,4.940787e-01,4.454511e-02,1.802216e-02,9.997287e-02,7.168002e-03,4.349081e-03,4.366169e-03,4.145783e-01,6.353947e+01,5.902777e+01,6.004179e+01,0.0,0.0,0.0,0.0,0.0,0.0,7.032073e+01,1.067439e+05,4.669613e+01,1.348631e+05,0.0,1.022955e+04,7.017818e+01,0.0,4.749506e+06,7.450372e+05,4.860748e+06,4.697343e+06,7.926355e+06,1.555991e+06,8.369454e+06,7.749780e+06,8.125145e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e

In [50]:
df.isna().sum()

Flow ID     0
Src IP      0
Src Port    0
Dst IP      0
Dst Port    0
           ..
Idle Std    0
Idle Max    0
Idle Min    0
Label       0
Attack      0
Length: 84, dtype: int64

In [51]:
def to_categorical_codes(column):
        return pd.Categorical(column).codes
    
if name=='x_iiot':
    columns_to_convert = [
          'Protocol', 'Service', 'is_syn_only', 'Is_SYN_ACK', 
           'is_pure_ack', 'is_with_payload', 'FIN or RST', 
        'Bad_checksum', 'is_SYN_with_RST', 'anomaly_alert']


    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
            
    df['class3'] = (df['class3'] == 'Attack').astype(int)
    df = df.applymap(lambda x: 0 if x == '-' else x)
    df = df.applymap(lambda x: 0 if x == '?' else x)
    df = df.applymap(lambda x: 0 if x == '#DIV/0!' else x)    
    df = df.applymap(lambda x: 0 if x == 'excel' else x)
    df = df.applymap(lambda x: 0 if x == 'aza' else x)
    df = df.applymap(lambda x: 0 if x == ' ' else x)
    
if name=='edge_iiot':
    columns_to_convert = ['mqtt.protoname','mqtt.topic','mqtt.conack.flags','mqtt.msg','http.request.method','dns.qry.name.len','arp.src.proto_ipv4',
                          'http.request.full_uri','http.file_data','http.request.version','arp.dst.proto_ipv4','http.request.uri.query','tcp.srcport','http.referer']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    
if name=='ccd_inid_modified':    
    columns_to_convert = ['splt_direction','splt_ps','splt_piat_ms','application_name','application_category_name','requested_server_name','client_fingerprint']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    df['traffic_type'] = (df['traffic_type'] == 'attack').astype(int)
    df.drop(['requested_server_name', 'client_fingerprint', 'server_fingerprint', 'user_agent', 'content_type'], axis=1, inplace=True)

if name=='cic_ton_iot_modified':
    df.drop(['datetime'], axis=1, inplace=True)


    

In [52]:
df = clean_dataset(df, flow_id_col=datesetInfo.flow_id_col, timestamp_col=datesetInfo.timestamp_col)

==>> original df.shape[0]: 13428595
==>> after drop na df.shape[0]: 13427562
==>> after drop_duplicates df.shape[0]: 13427518


In [53]:
df[datesetInfo.src_ip_col] = df[datesetInfo.src_ip_col].apply(str)
df[datesetInfo.src_port_col] = df[datesetInfo.src_port_col].apply(str)
df[datesetInfo.dst_ip_col] = df[datesetInfo.dst_ip_col].apply(str)
df[datesetInfo.dst_port_col] = df[datesetInfo.dst_port_col].apply(str)

In [54]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack
0,192.168.100.150-192.168.100.3-54114-44581-6,192.168.100.150,54114.0,192.168.100.3,44581.0,6.0,21/05/2018 08:34:13 PM,64465.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,31.024587,6.446500e+04,0.000000e+00,64465.0,64465.0,0.0,0.000000e+00,0.0,0.0,64465.0,64465.0,0.0,64465.0,64465.0,0.0,0.0,0.0,0.0,0.0,44.0,0.000000,31.024587,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance
1,192.168.100.147-192.168.100.7-1649-80-6,192.168.100.147,1649.0,192.168.100.7,80.0,6.0,04/06/2018 05:38:04 PM,22646986.0,3.0,1.0,300.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,100.0,0.000000,17.662394,0.176624,7.548995e+06,5.561972e+06,13839423.0,3281987.0,19364999.0,9.682500e+06,13839423.0,5525576.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,20.0,0.132468,0.044156,100.0,100.0,100.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,300.0,1.0,100.0,-1.0,512.0,3.0,0.0,3281987.0,0.000000e+00,3281987.0,3281987.0,9682499.5,5.878778e+06,13839423.0,5525576.0,1,DDoS
2,192.168.100.148-192.168.100.3-59032-80-6,192.168.100.148,59032.0,192.168.100.3,80.0,6.0,04/06/2018 01:41:34 PM,153463.0,1.0,2.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,50.0,70.710678,651.622867,19.548686,7.673150e+04,7.671755e+04,130979.0,22484.0,0.0,0.000000e+00,0.0,0.0,22484.0,22484.0,0.0,22484.0,22484.0,0.0,0.0,0.0,0.0,20.0,44.0,6.516229,13.032457,0.0,100.0,50.0,57.735027,3333.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,66.666667,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,100.0,-1.0,29200.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance
3,192.168.100.149-192.168.100.7-2820-80-17,192.168.100.149,2820.0,192.168.100.7,80.0,17.0,04/06/2018 02:29:28 PM,24915470.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.200679,6.228868e+06,2.337803e+06,9613711.0,4568562.0,20138439.0,6.712813e+06,9613711.0,4568562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,8.0,0.160543,0.040136,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,4777031.0,0.000000e+00,4777031.0,4777031.0,7784938.5,2.586275e+06,9613711.0,5956166.0,1,DDoS
4,192.168.100.150-192.168.100.5-5146-80-17,192.168.100.150,5146.0,192.168.100.5,80.0,17.0,04/06/2018 06:17:12 PM,46437692.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.193808,5.804712e+06,3.854238e+06,14796053.0,3089320.0,42448141.0,6.064020e+06,14796053.0,3089320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,8.0,0.172274,0.021534,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,9597806.5,7.235516e+06,14714089.0,4481524.0,9080693.0,5.048133e+06,147

In [55]:
df.describe()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,13427518.0,13427518.0,13427518.0,13427518.0,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07
mean,1.052062e+01,2.392931e+07,6.245421e+00,1.942021e+00,1.139305e+03,5.868635e+02,4.892058e+01,3.014441e+01,3.967842e+01,1.022045e+01,4.950924e+01,2.472911e+01,3.422295e+01,1.493991e+01,1.749702e+04,3.561901e+03,5.043776e+06,2.544952e+06,8.633147e+06,2.983166e+06,1.869024e+07,5.386039e+06,7.610522e+06,3.626387e+06,4.194444e+06,2.680920e+06,1.432961e+06,3.704618e+06,1.581323e+06,0.0,3.232168e-04,0.0,5.138701e-05,7.799179e+01,3.397108e+01,6.972383e+02,2.864662e+03,2.459343e+01,5.111762e+01,3.857025e+01,1.232192e+01,2.093946e+03,1.845836e-03,5.767571e-01,1.984134e-03,3.232168e-04,1.003276e-02,5.138701e-05,1.891638e-05,1.906533e-05,1.508414e-01,4.718783e+01,3.967842e+01,3.422295e+01,0.0,0.0,0.0,0.0,0.0,0.0,6.245421e+00,1.139305e+03,1.942021e+00,5.868635e+02,-1.0,4.367750e+03,2.560943e+00,0.0,2.058155e+06,1.024744e+05,2.131938e+06,1.985306e+06,7.060359e+06,7.965978e+05,7.842334e+06,6.424130e+06,9.933577e-01
std,5.412682e+00,1.694539e+07,7.032354e+01,4.669800e+01,1.067482e+05,1.348685e+05,9.352652e+01,4.703042e+01,5.902855e+01,3.853029e+01,1.017542e+02,4.526081e+01,6.004267e+01,4.113192e+01,1.029434e+06,5.360078e+04,4.717517e+06,3.535817e+06,7.754182e+06,4.338945e+06,1.473619e+07,5.208608e+06,7.414965e+06,4.799894e+06,1.086291e+07,7.039797e+06,5.021545e+06,9.384669e+06,6.143300e+06,0.0,1.797533e-02,0.0,7.168289e-03,1.170487e+03,8.248857e+02,2.354497e+04,3.522766e+04,4.356331e+01,1.185575e+02,5.446903e+01,4.406944e+01,8.231854e+05,4.292353e-02,4.940732e-01,4.449941e-02,1.797533e-02,9.965992e-02,7.168289e-03,4.349255e-03,4.366344e-03,4.145358e-01,6.353914e+01,5.902855e+01,6.004267e+01,0.0,0.0,0.0,0.0,0.0,0.0,7.032354e+01,1.067482e+05,4.669800e+01,1.348685e+05,0.0,1.022989e+04,7.018099e+01,0.0,4.749661e+06,7.450665e+05,4.860906e+06,4.697498e+06,7.926421e+06,1.556037e+06,8.369495e+06,7.749877e+06,8.122936e-02
min,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00

In [56]:
_,var_dropped, corr_dropped = feature_analysis_pipeline(df=df, drop_columns=datesetInfo.drop_columns,label_col=datesetInfo.label_col)
var_dropped, corr_dropped

(['Fwd PSH Flags',
  'Fwd URG Flags',
  'Fwd Byts/b Avg',
  'Fwd Pkts/b Avg',
  'Fwd Blk Rate Avg',
  'Bwd Byts/b Avg',
  'Bwd Pkts/b Avg',
  'Bwd Blk Rate Avg',
  'Init Fwd Win Byts',
  'Fwd Seg Size Min'],
 {'Active Max',
  'Active Mean',
  'Bwd Header Len',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd IAT Tot',
  'Bwd PSH Flags',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Bwd Pkt Len Std',
  'Bwd URG Flags',
  'CWE Flag Count',
  'Flow Duration',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow Pkts/s',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd IAT Mean',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Min',
  'Idle Max',
  'Idle Mean',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Size Avg',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts'})

In [74]:
print(f"==>> type(var_dropped): {type(var_dropped)}")
print(f"==>> type(corr_dropped): {type(corr_dropped)}")

==>> type(var_dropped): <class 'list'>
==>> type(corr_dropped): <class 'set'>


In [78]:
var_dropped = set(var_dropped)
weak_columns = var_dropped.union(set(corr_dropped))
weak_columns

{'Active Max',
 'Active Mean',
 'Bwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Header Len',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Tot',
 'Bwd PSH Flags',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Std',
 'Bwd Pkts/b Avg',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Flow Duration',
 'Flow IAT Max',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow Pkts/s',
 'Fwd Blk Rate Avg',
 'Fwd Byts/b Avg',
 'Fwd Header Len',
 'Fwd IAT Max',
 'Fwd IAT Mean',
 'Fwd PSH Flags',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Min',
 'Fwd Pkts/b Avg',
 'Fwd Seg Size Min',
 'Fwd URG Flags',
 'Idle Max',
 'Idle Mean',
 'Init Fwd Win Byts',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Size Avg',
 'Subflow Fwd Pkts',
 'Tot Bwd Pkts',
 'Tot Fwd Pkts',
 'TotLen Bwd Pkts',
 'TotLen Fwd Pkts'}

### Attacks Types

In [57]:
df[datesetInfo.class_col].unique()

array(['Reconnaissance', 'DDoS', 'DoS', 'Benign', 'Theft'], dtype=object)

In [58]:
# df[class_col] = df[class_col].replace({"BENIGN": "Benign",
#                                        "DDoS": "ddos",
#                                        "Web Attack � Brute Force": "bruteforce",
#                                        "Web Attack � XSS": "xss"})

In [59]:
classes = df[datesetInfo.class_col].unique()

### Sorting (optional)

In [60]:
if with_sort_timestamp and datesetInfo.timestamp_col:
    df[datesetInfo.timestamp_col] = pd.to_datetime(df[datesetInfo.timestamp_col].str.strip(), format=timestamp_format)
    df.sort_values(datesetInfo.timestamp_col, inplace= True)

### Encoding Attacks into integers

In [61]:
df, labels_names = one_dataset_class_num_col(df, datesetInfo.class_num_col, datesetInfo.class_col)

==>> labels_names: {np.int64(0): np.str_('Benign'), np.int64(1): np.str_('DDoS'), np.int64(2): np.str_('DoS'), np.int64(3): np.str_('Reconnaissance'), np.int64(4): np.str_('Theft')}


In [62]:
df.groupby(datesetInfo.class_col).size()

Attack
Benign              89190
DDoS              4913860
DoS               4909359
Reconnaissance    3513408
Theft                1701
dtype: int64

### Undersampling classes (optional)

In [63]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(datesetInfo.class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

In [64]:
if with_undersample_classes:
    # Get the classes with the highest number of records (you can choose how many to undersample)
    classes_to_undersample = class_counts_sorted.index[:2]

    # Undersample the classes with the highest number of records
    dfs = []
    for class_label in class_counts_sorted.index:
        print(f"==>> class_label: {class_label}")
        if class_label in classes_to_undersample:
            class_df = df[df[datesetInfo.class_col] == class_label]
            undersampled_df = class_df.sample(frac=0.5)  # Specify the fraction of samples to keep
            dfs.append(undersampled_df)
        else:
            dfs.append(df[df[datesetInfo.class_col] == class_label])

    df = []
    # Optional: shuffle the undersampled DataFrame
    df = pd.concat(dfs).sample(frac=1).reset_index(drop=True)


In [65]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(datesetInfo.class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

### saving labels encodings and datasets properties

In [66]:
with open(folder_path + '/labels_names.pkl', 'wb') as f:
    pickle.dump([labels_names, classes], f)

In [67]:
df[datesetInfo.label_col].unique()

array([1, 0])

In [68]:
total_count = len(df)

properties = {
    "name": name,
    "length": total_count,
}

num_benign = len(df[df[datesetInfo.label_col] == 0])
num_attack = len(df[df[datesetInfo.label_col] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df[datesetInfo.class_col].unique())

properties["weak_columns"] = list(weak_columns) 

with open(folder_path + '/df_properties.json', 'w') as f:
    json.dump(properties, f)
    
properties


{'name': 'cic_bot_iot',
 'length': 13427518,
 'num_benign': 89190,
 'percentage_of_benign_records': 0.664232958019494,
 'num_attack': 13338328,
 'percentage_of_attack_records': 99.3357670419805,
 'attacks': ['Reconnaissance', 'DDoS', 'DoS', 'Benign', 'Theft']}

In [69]:
# graphs_properties_path = folder_path + '/graphs_properties.json'
# G = nx.from_pandas_edgelist(
#     df,
#     source=src_ip_col,
#     target=dst_ip_col,
#     create_using=nx.DiGraph()
# )
# calculate_graph_measures(G, graphs_properties_path)

In [70]:
df.to_parquet(output_path)

In [73]:
df.shape

(13427518, 85)

In [71]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,...,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
0,192.168.100.150-192.168.100.3-54114-44581-6,192.168.100.150,54114.0,192.168.100.3,44581.0,6.0,21/05/2018 08:34:13 PM,64465.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,31.024587,6.446500e+04,0.000000e+00,64465.0,64465.0,0.0,0.000000e+00,0.0,0.0,64465.0,64465.0,0.0,64465.0,64465.0,0.0,0.0,0.0,0.0,0.0,44.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance,3
1,192.168.100.147-192.168.100.7-1649-80-6,192.168.100.147,1649.0,192.168.100.7,80.0,6.0,04/06/2018 05:38:04 PM,22646986.0,3.0,1.0,300.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,100.0,0.000000,17.662394,0.176624,7.548995e+06,5.561972e+06,13839423.0,3281987.0,19364999.0,9.682500e+06,13839423.0,5525576.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,20.0,0.132468,...,100.0,100.0,100.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,300.0,1.0,100.0,-1.0,512.0,3.0,0.0,3281987.0,0.000000e+00,3281987.0,3281987.0,9682499.5,5.878778e+06,13839423.0,5525576.0,1,DDoS,1
2,192.168.100.148-192.168.100.3-59032-80-6,192.168.100.148,59032.0,192.168.100.3,80.0,6.0,04/06/2018 01:41:34 PM,153463.0,1.0,2.0,0.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,50.0,70.710678,651.622867,19.548686,7.673150e+04,7.671755e+04,130979.0,22484.0,0.0,0.000000e+00,0.0,0.0,22484.0,22484.0,0.0,22484.0,22484.0,0.0,0.0,0.0,0.0,20.0,44.0,6.516229,...,0.0,100.0,50.0,57.735027,3333.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,66.666667,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,100.0,-1.0,29200.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1,Reconnaissance,3
3,192.168.100.149-192.168.100.7-2820-80-17,192.168.100.149,2820.0,192.168.100.7,80.0,17.0,04/06/2018 02:29:28 PM,24915470.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.200679,6.228868e+06,2.337803e+06,9613711.0,4568562.0,20138439.0,6.712813e+06,9613711.0,4568562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,8.0,0.160543,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,4777031.0,0.000000e+00,4777031.0,4777031.0,7784938.5,2.586275e+06,9613711.0,5956166.0,1,DDoS,1
4,192.168.100.150-192.168.100.5-5146-80-17,192.168.100.150,5146.0,192.168.100.5,80.0,17.0,04/06/2018 06:17:12 PM,46437692.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.193808,5.804712e+06,3.854238e+06,14796053.0,3089320.0,42448141.0,6.064020e+06,14796053.0,3089320.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,8.0,0.172274,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,-1.0,-1.0,0.0,0.0,9597806.5,7.235516e+06,14714089.0,4481524.0,9080693.0,5.048133e+06,14796053.0,5230723.0,1,

In [72]:
df.describe()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Class
count,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,13427518.0,13427518.0,13427518.0,13427518.0,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,13427518.0,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07,1.342752e+07
mean,1.052062e+01,2.392931e+07,6.245421e+00,1.942021e+00,1.139305e+03,5.868635e+02,4.892058e+01,3.014441e+01,3.967842e+01,1.022045e+01,4.950924e+01,2.472911e+01,3.422295e+01,1.493991e+01,1.749702e+04,3.561901e+03,5.043776e+06,2.544952e+06,8.633147e+06,2.983166e+06,1.869024e+07,5.386039e+06,7.610522e+06,3.626387e+06,4.194444e+06,2.680920e+06,1.432961e+06,3.704618e+06,1.581323e+06,0.0,3.232168e-04,0.0,5.138701e-05,7.799179e+01,3.397108e+01,6.972383e+02,2.864662e+03,2.459343e+01,5.111762e+01,3.857025e+01,1.232192e+01,2.093946e+03,1.845836e-03,5.767571e-01,1.984134e-03,3.232168e-04,1.003276e-02,5.138701e-05,1.891638e-05,1.906533e-05,1.508414e-01,4.718783e+01,3.967842e+01,3.422295e+01,0.0,0.0,0.0,0.0,0.0,0.0,6.245421e+00,1.139305e+03,1.942021e+00,5.868635e+02,-1.0,4.367750e+03,2.560943e+00,0.0,2.058155e+06,1.024744e+05,2.131938e+06,1.985306e+06,7.060359e+06,7.965978e+05,7.842334e+06,6.424130e+06,9.933577e-01,1.882672e+00
std,5.412682e+00,1.694539e+07,7.032354e+01,4.669800e+01,1.067482e+05,1.348685e+05,9.352652e+01,4.703042e+01,5.902855e+01,3.853029e+01,1.017542e+02,4.526081e+01,6.004267e+01,4.113192e+01,1.029434e+06,5.360078e+04,4.717517e+06,3.535817e+06,7.754182e+06,4.338945e+06,1.473619e+07,5.208608e+06,7.414965e+06,4.799894e+06,1.086291e+07,7.039797e+06,5.021545e+06,9.384669e+06,6.143300e+06,0.0,1.797533e-02,0.0,7.168289e-03,1.170487e+03,8.248857e+02,2.354497e+04,3.522766e+04,4.356331e+01,1.185575e+02,5.446903e+01,4.406944e+01,8.231854e+05,4.292353e-02,4.940732e-01,4.449941e-02,1.797533e-02,9.965992e-02,7.168289e-03,4.349255e-03,4.366344e-03,4.145358e-01,6.353914e+01,5.902855e+01,6.004267e+01,0.0,0.0,0.0,0.0,0.0,0.0,7.032354e+01,1.067482e+05,4.669800e+01,1.348685e+05,0.0,1.022989e+04,7.018099e+01,0.0,4.749661e+06,7.450665e+05,4.860906e+06,4.697498e+06,7.926421e+06,1.556037e+06,8.369495e+06,7.749877e+06,8.122936e-02,8.005760e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+0